In [1]:
import pandas as pd
import tabula 
from urllib import request
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urljoin
import re
from PyPDF2 import PdfFileReader
import matplotlib.pyplot as plt
import yfinance as yf

In [2]:
url = "https://petrobras.com.br/pt/nossas-atividades/precos-de-venda-as-distribuidoras/gasolina-e-diesel/"
html = urlopen(url)
bs = BeautifulSoup(html, "html.parser")

# DIESEL
links = []
for x in bs.find_all("a", href = True):
    if re.search(re.compile("diesel", flags = re.IGNORECASE), str(x)):
        links.append(x["href"])

In [3]:
root = "https://petrobras.com.br/"
link_final_diesel = links[2][9:]
link_diesel = root + link_final_diesel

In [ ]:
pag_pdf = tabula.read_pdf(link_diesel, pages = "all", multiple_tables=True)

In [ ]:
agrupado = pd.DataFrame()
for dataframe in range(0,len(pag_pdf)):
    
    if len(pag_pdf[dataframe]) > 25:        
        df = pag_pdf[dataframe]
        
        for line_num, line in enumerate(pag_pdf[dataframe].iloc[:,0]):
            if line == "LOCAL":
                linha_datas = df.iloc[line_num, :]
        
        df2 = df.dropna(subset = [df.columns[0]])
#         print(linha_datas)
#         print(df2.head(20))
        try:
            df2.columns = linha_datas
        except:
            print(f"erro página {dataframe}")
            continue
        df2 = df2.reset_index(drop = True)
        df2.drop(0, axis = 0, inplace = True)
        df3 = pd.melt(df2, id_vars = df2.columns[:2], value_vars = df2.columns[2:])
        df3.columns = ["LOCAL", "MODALIDADE", "DATA", "VALOR"]
        agrupado = pd.concat([agrupado, df3], axis = 0)

In [ ]:
agrupado.dropna(subset = ["VALOR"], inplace=True)

agrupado.loc[:,"VALOR"] = agrupado["VALOR"].str.replace(".", "")
agrupado.loc[:,"VALOR"] = agrupado["VALOR"].apply(lambda x: float(x.replace(",", "."))/1000)
agrupado.loc[:,"DATA"] = agrupado["DATA"].apply(lambda x: pd.to_datetime(x, format = "%d.%m.%Y"))

In [60]:
agrupado_canoas = agrupado[agrupado["LOCAL"] == "Canoas (RS)"]

In [62]:
agrupado_canoas_exa = agrupado_canoas[agrupado_canoas["MODALIDADE"] == "EXA"]

In [ ]:
agrupado_canoas_exa["PRODUTO"] = 0

for line_num, line in enumerate(agrupado_canoas_exa["DATA"]):
    data_corrente = line
    try:
        data_proxima = agrupado_canoas_exa.iloc[line_num + 1, 2]
        if data_corrente < data_proxima:
            agrupado_canoas_exa.iloc[line_num, 4] = "S500"
    except:
        continue